# Solving Linear Programs with Gurobi Optimizer

This notebook contains two examples of how to formulate and solve linear programs using Python and Gurobi. The first example is the "Bag Production" problem from class. The second example is the "Inventory Management" example from class.

## Example 1: Bag Production

Recall that the linear program for the bag production problem is given by 

\begin{align}
\underset{x_S, x_D}{\text{max}} \;\; & 10 x_s + 8 x_D\\
\text{s.t.} \;\; &\frac{7}{10} x_S + x_D \le 630 \\
& \frac{1}{2} x_S + \frac{5}{6} x_D \le 600 \\
& x_S + \frac{2}{3} x_D \le 708 \\
& \frac{1}{10}x_S + \frac{1}{4} x_D \le 135  \\
& x_S, x_D \ge 0.
\end{align}

To solve the LP above, we first load the Gurobi module into Python.

In [2]:
from gurobipy import *

We will now formulate the LP using functions provided by the Gurobi module.

In [4]:
# Construct a 'blank' model. 
mod = Model()

# Define decision variables. We will use variable names 'S' and 'D' for simplicity.
S = mod.addVar()
D = mod.addVar()

# Construct constraints.
cutting_con = mod.addConstr((7/10)*S + D  <= 630)
sewing_con =  mod.addConstr((1/2)*S + (5/6)*D  <= 600)
finishing_con =  mod.addConstr(S + (2/3)*D  <= 708)
inspecting_con = mod.addConstr((1/10)*S + (1/4)*D  <= 135)

# Add non-negativity constraints.
mod.addConstr(S >= 0.0)
mod.addConstr(D >= 0.0)

# Create the objective function, and set it to be maximized.
mod.setObjective(10*S + 9*D, GRB.MAXIMIZE)

Using license file /Users/vagrant/gurobi.lic
Academic license - for non-commercial use only


We now push the updated constraints and objective to the blank model.

In [5]:
mod.update()

Now that the model is set up, we can call Gurobi to solve it.

In [6]:
mod.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 6 rows, 2 columns and 10 nonzeros
Model fingerprint: 0x7cdc10a6
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 7e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.02s
Presolved: 4 rows, 2 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+03   4.791138e+01   0.000000e+00      0s
       2    7.6680000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective  7.668000000e+03


After we solve the problem, we can retrieve the optimal solution found by Gurobi and the associated optimal value.

In [7]:
# Extract the solution status. 
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
# Extract the optimal values of the decision variables.
S_opt = S.x
D_opt = D.x

print("Number of Standard bags: ", S_opt)
print("Number of Deluxe bags: ", D_opt)

# Retrieve the optimal value.
opt_val = mod.objval
print("Optimal profit:", opt_val)

Solved to optimality
Number of Standard bags:  539.9999999999999
Number of Deluxe bags:  252.0000000000001
Optimal profit: 7668.0


## Example 2: Product Delivery

For the product delivery problem, it may be impractical to write the constraints explicitly if the number of warehouses and/or customers is large. Instead, we will import the data from .csv files. Recall that the linear program is given by


\begin{align}
\underset{{\bf x}}{\text{min}} \;\; & \sum_{i=1}^m \sum_{j=1}^n c_{ij} x_{ij} \\
\text{s.t.} & \sum_{i=1}^m x_{ij} \ge d_j, \quad j = 1,2,\ldots,n, \\
& \sum_{j=1}^n x_{ij} \le s_i, \quad i = 1,2,\ldots,m, \\
& x_{ij} \ge 0, \quad i = 1,2,\ldots,m, \; j = 1,2,\ldots,n.
\end{align}





First, we read in the model parameters using the numpy package.

In [20]:
from numpy import genfromtxt
demand = genfromtxt('demand.csv', delimiter=',')
supply = genfromtxt('supply.csv', delimiter=',')
cost = genfromtxt('cost.csv', delimiter=',')

We now formulate the constraints and objective.

In [21]:
# Define the data 
m = 10;
n = 500;

# Initialize the model 
mod = Model()

# Define variables 
# mod.addVars(m,n) will create variables where rows are indexed from 0 to m-1 and columns are indexed from 0 to n-1 
x = mod.addVars(m,n)

# Constraint: All demands must be satisfied
demand_con = {}
for j in range(n):
    demand_con[j] = mod.addConstr(sum(x[i,j] for i in range(m)) >= demand[j])
    
# Constraint: Total delivery from each warehouse cannot exceed available supply
supply_con = {}
for i in range(m):
    supply_con[j] = mod.addConstr(sum(x[i,j] for j in range(n)) <= supply[i])
    
# Nonnegativity constraints:
for i in range(m):
    for j in range(n):
        mod.addConstr(x[i,j] >= 0.0)

# Construct objective
mod.setObjective(sum(cost[i][j] * x[i,j] for i in range(m) for j in range(n)), GRB.MINIMIZE)

Note that the constraints are written in a form that closely resemble the mathematical expressions in the LP formulation -- this readability is one of the advantages of using the Gurobi package in Python to build and solve optimization models. 

Next, we update and solve the model.

In [22]:
# Update and solve
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 5510 rows, 5000 columns and 15000 nonzeros
Model fingerprint: 0x15cb881c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+02]
Presolve removed 5000 rows and 0 columns
Presolve time: 0.01s
Presolved: 510 rows, 5000 columns, 10000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.142678e+01   0.000000e+00      0s
     577    2.5642351e+01   0.000000e+00   0.000000e+00      0s

Solved in 577 iterations and 0.02 seconds
Optimal objective  2.564235109e+01


Observe that the output provides the number of simplex algorithm iterations taken to solve the LP.  

As in Example 1, we can now retrieve the optimal solution and optimal value.

In [23]:
# Retrieve optimal value and optimal solution
opt_val = mod.objval
print("Optimal cost:",opt_val)

x_opt = [x[i,j].x for i in range(m) for j in range(n)]
print(x_opt)

Optimal cost: 25.642351091747752
[0.0, 0.0, 0.79442612, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.65808973, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.62451576, 0.18313888, 0.0, 0.89630676, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99956897, 0.55641985, 0.0, 0.0, 0.0, 0.0, 0.0, 0.80288216, 0.0, 0.0, 0.0, 0.65559661, 0.0, 0.0, 0.0, 0.0, 0.27034121, 0.0, 0.0, 0.0, 0.0, 0.49900153, 0.0, 0.0, 0.78831782, 0.0, 0.47949353, 0.0, 0.68921674, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.68327286, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34346244, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.74789833, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.83729652, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6044689, 0.71418836, 0.25696758, 0.0, 0.65410904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29523295, 0

# Summary

These are just two simple examples of how to solve linear programs using Gurobi. The general principles used above can be extended to much larger and more complex optimization models.